# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fd6385e9bb0>
├── 'a' --> tensor([[ 0.8698, -1.1469, -0.5422],
│                   [-0.4405, -0.5759,  2.5385]])
└── 'x' --> <FastTreeValue 0x7fd6385e9d30>
    └── 'c' --> tensor([[-1.9124,  3.0756,  0.1887,  0.1893],
                        [ 1.3464, -0.8070,  0.3804, -0.2231],
                        [ 0.9050,  0.6029,  0.0937,  0.2743]])

In [4]:
t.a

tensor([[ 0.8698, -1.1469, -0.5422],
        [-0.4405, -0.5759,  2.5385]])

In [5]:
%timeit t.a

63.2 ns ± 0.0165 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fd6385e9bb0>
├── 'a' --> tensor([[-0.3615,  0.7360,  1.0103],
│                   [-0.1178, -0.3833,  0.4855]])
└── 'x' --> <FastTreeValue 0x7fd6385e9d30>
    └── 'c' --> tensor([[-1.9124,  3.0756,  0.1887,  0.1893],
                        [ 1.3464, -0.8070,  0.3804, -0.2231],
                        [ 0.9050,  0.6029,  0.0937,  0.2743]])

In [7]:
%timeit t.a = new_value

63.8 ns ± 0.0129 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.8698, -1.1469, -0.5422],
               [-0.4405, -0.5759,  2.5385]]),
    x: Batch(
           c: tensor([[-1.9124,  3.0756,  0.1887,  0.1893],
                      [ 1.3464, -0.8070,  0.3804, -0.2231],
                      [ 0.9050,  0.6029,  0.0937,  0.2743]]),
       ),
)

In [10]:
b.a

tensor([[ 0.8698, -1.1469, -0.5422],
        [-0.4405, -0.5759,  2.5385]])

In [11]:
%timeit b.a

51 ns ± 0.0497 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.2217, -0.9715, -0.6403],
               [-1.0826, -0.8373, -0.9864]]),
    x: Batch(
           c: tensor([[-1.9124,  3.0756,  0.1887,  0.1893],
                      [ 1.3464, -0.8070,  0.3804, -0.2231],
                      [ 0.9050,  0.6029,  0.0937,  0.2743]]),
       ),
)

In [13]:
%timeit b.a = new_value

479 ns ± 0.232 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

904 ns ± 12.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 8.64 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

134 µs ± 297 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

138 µs ± 489 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fd6385e9be0>
├── 'a' --> tensor([[[ 0.8698, -1.1469, -0.5422],
│                    [-0.4405, -0.5759,  2.5385]],
│           
│                   [[ 0.8698, -1.1469, -0.5422],
│                    [-0.4405, -0.5759,  2.5385]],
│           
│                   [[ 0.8698, -1.1469, -0.5422],
│                    [-0.4405, -0.5759,  2.5385]],
│           
│                   [[ 0.8698, -1.1469, -0.5422],
│                    [-0.4405, -0.5759,  2.5385]],
│           
│                   [[ 0.8698, -1.1469, -0.5422],
│                    [-0.4405, -0.5759,  2.5385]],
│           
│                   [[ 0.8698, -1.1469, -0.5422],
│                    [-0.4405, -0.5759,  2.5385]],
│           
│                   [[ 0.8698, -1.1469, -0.5422],
│                    [-0.4405, -0.5759,  2.5385]],
│           
│                   [[ 0.8698, -1.1469, -0.5422],
│                    [-0.4405, -0.5759,  2.5385]]])
└── 'x' --> <FastTreeValue 0x7fd594f04c40>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.2 µs ± 87 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fd594ee7460>
├── 'a' --> tensor([[ 0.8698, -1.1469, -0.5422],
│                   [-0.4405, -0.5759,  2.5385],
│                   [ 0.8698, -1.1469, -0.5422],
│                   [-0.4405, -0.5759,  2.5385],
│                   [ 0.8698, -1.1469, -0.5422],
│                   [-0.4405, -0.5759,  2.5385],
│                   [ 0.8698, -1.1469, -0.5422],
│                   [-0.4405, -0.5759,  2.5385],
│                   [ 0.8698, -1.1469, -0.5422],
│                   [-0.4405, -0.5759,  2.5385],
│                   [ 0.8698, -1.1469, -0.5422],
│                   [-0.4405, -0.5759,  2.5385],
│                   [ 0.8698, -1.1469, -0.5422],
│                   [-0.4405, -0.5759,  2.5385],
│                   [ 0.8698, -1.1469, -0.5422],
│                   [-0.4405, -0.5759,  2.5385]])
└── 'x' --> <FastTreeValue 0x7fd592728f40>
    └── 'c' --> tensor([[-1.9124,  3.0756,  0.1887,  0.1893],
                        [ 1.3464, -0.8070,  0.3804, -0.2231],
                 

In [23]:
%timeit t_cat(trees)

29.6 µs ± 105 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.4 µs ± 60.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-1.9124,  3.0756,  0.1887,  0.1893],
                       [ 1.3464, -0.8070,  0.3804, -0.2231],
                       [ 0.9050,  0.6029,  0.0937,  0.2743]],
              
                      [[-1.9124,  3.0756,  0.1887,  0.1893],
                       [ 1.3464, -0.8070,  0.3804, -0.2231],
                       [ 0.9050,  0.6029,  0.0937,  0.2743]],
              
                      [[-1.9124,  3.0756,  0.1887,  0.1893],
                       [ 1.3464, -0.8070,  0.3804, -0.2231],
                       [ 0.9050,  0.6029,  0.0937,  0.2743]],
              
                      [[-1.9124,  3.0756,  0.1887,  0.1893],
                       [ 1.3464, -0.8070,  0.3804, -0.2231],
                       [ 0.9050,  0.6029,  0.0937,  0.2743]],
              
                      [[-1.9124,  3.0756,  0.1887,  0.1893],
                       [ 1.3464, -0.8070,  0.3804, -0.2231],
                       [ 0.9050,  0.6029,  0.0937,  0.2743]],

In [26]:
%timeit Batch.stack(batches)

78 µs ± 157 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-1.9124,  3.0756,  0.1887,  0.1893],
                      [ 1.3464, -0.8070,  0.3804, -0.2231],
                      [ 0.9050,  0.6029,  0.0937,  0.2743],
                      [-1.9124,  3.0756,  0.1887,  0.1893],
                      [ 1.3464, -0.8070,  0.3804, -0.2231],
                      [ 0.9050,  0.6029,  0.0937,  0.2743],
                      [-1.9124,  3.0756,  0.1887,  0.1893],
                      [ 1.3464, -0.8070,  0.3804, -0.2231],
                      [ 0.9050,  0.6029,  0.0937,  0.2743],
                      [-1.9124,  3.0756,  0.1887,  0.1893],
                      [ 1.3464, -0.8070,  0.3804, -0.2231],
                      [ 0.9050,  0.6029,  0.0937,  0.2743],
                      [-1.9124,  3.0756,  0.1887,  0.1893],
                      [ 1.3464, -0.8070,  0.3804, -0.2231],
                      [ 0.9050,  0.6029,  0.0937,  0.2743],
                      [-1.9124,  3.0756,  0.1887,  0.1893],
                   

In [28]:
%timeit Batch.cat(batches)

142 µs ± 249 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

310 µs ± 2.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
